In [317]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

df = pd.read_excel("C:\\Users\\Admin\\Desktop\\DATN\\data.xlsx")
df


,ResponseId,MainBranch,Age,Country,EdLevel,WorkExp,CompTotal,RemoteWork,Employment,OrgSize,...,OfficeStackSyncHaveWorkedWith,SOComm,AISelect,AISent,AIBen,AIAcc,AIToolCurrently Using,TBranch,ICorPM,BranchGroup
0,393,I am a developer by profession,35-44,United Kingdom Of Great Britain And Northern I...,Professional,18,126420.0,Hybrid,Employ,Large,...,Slack,No,Yes,Unfavorable,Improve accuracy in coding,Somewhat distrust,Writing code,Yes,People manager,Developer
1,417,I am a developer by profession,35-44,Brazil,Master,17,170000.0,Remote,Employ-Freelance,Medium,...,Google Meet-Telegram-Whatsapp-Zoom,No,Yes,Favorable,Increase productivity,Somewhat distrust,Writing code-Documenting code-Testing code,Yes,Individual contributor,Developer
2,424,I am a developer by profession,25-34,Italy,No Degree,4,62000.0,Remote,Employ-Freelance,Medium,...,Google Meet-Mattermost-Slack,Yes,Yes,Unfavorable,Increase productivity,Highly distrust,Writing code,Yes,Individual contributor,Developer
3,444,"I am not primarily a developer, but I write co...",35-44,United Kingdom Of Great Britain And Northern I...,Bachelor,20,127000.0,Remote,Employ,Enterprise,...,Google Meet-Microsoft Teams-Skype-Slack-Telegr...,Somewhat,Yes,Favorable,Increase productivity-Greater efficiency-Speed...,Somewhat trust,Writing code-Search for answers-Generating con...,Yes,People manager,Semi-technical
4,445,I am a developer by profession,25-34,United Kingdom Of Great Britain And Northern I...,Bachelor,10,115000.0,Remote,Employ-Student,Small,...,Slack-Zoom,Neutral,Yes,Favorable,Speed up learning,Neither trust nor distrust,Learning about a codebase-Writing code,Yes,Individual contributor,Developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,65106,I am a developer by profession,25-34,United States Of America,Bachelor,8,155000.0,Hybrid,Employ,Enterprise,...,Symphony-Zoom,Somewhat,Yes,Favorable,Increase productivity-Greater efficiency,Somewhat trust,Writing code-Debugging and getting help-Testin...,Yes,Individual contributor,Developer
10101,65152,I am a developer by profession,35-44,Estonia,Master,14,90000.0,Hybrid,Employ,Large,...,Slack-Zoom,Neutral,Yes,Favorable,Increase productivity-Greater efficiency,Somewhat distrust,Writing code-Generating content or synthetic data,Yes,Individual contributor,Developer
10102,65164,I am a developer by profession,18-24,Germany,Associate,3,46000.0,Hybrid,Employ,Small,...,Discord-Microsoft Teams-Telegram-Whatsapp,Neutral,Yes,Favorable,Increase productivity-Greater efficiency-Impro...,Somewhat trust,Writing code-Documenting code-Debugging and ge...,Yes,Individual contributor,Developer
10103,65167,I am a developer by profession,25-34,Spain,Bachelor,8,81600.0,Remote,Employ,Large,...,Google Chat-Google Meet-Microsoft Teams-Skype-...,No,Yes,Very favorable,Increase productivity-Greater efficiency-Speed...,Somewhat trust,Learning about a codebase-Debugging and gettin...,Yes,Individual contributor,Developer


In [318]:
# chọn cột cần thiết
df_model = df[['DevType', 'Country', 'EdLevel', 'WorkExp', 'CompTotal', 'RemoteWork', 'OrgSize']].dropna()

# Giữ dữ liệu có lương hợp lý (USD/năm)
df_model = df_model[(df_model['CompTotal'] > 1000) & (df_model['CompTotal'] < 5000000000000000000000)]

# Tách X, y
X = df_model.drop(columns='CompTotal')
y = df_model['CompTotal']

# Xử lý số và phân loại
cot_so = ['WorkExp']
cot_phan_loai = ['DevType', 'Country', 'EdLevel', 'RemoteWork']

xu_ly = ColumnTransformer([
    ('so', SimpleImputer(strategy='mean'), cot_so),
    ('phan_loai', Pipeline([
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('encode', OneHotEncoder(handle_unknown='ignore'))
    ]), cot_phan_loai)
])

# Pipeline và model
pipeline = Pipeline([
    ('xu_ly', xu_ly),
    ('model', LinearRegression())
])

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Đánh giá mô hình 
df_model['LuongDuDoan'] = pipeline.predict(X)

# Lọc bỏ các dòng có lương dự đoán không hợp lí
df_model = df_model[(df_model['LuongDuDoan'] > 100) & (df_model['LuongDuDoan'] < 5000000000000000000000)]

# Tổng hợp kết quả theo ngành + hình thức làm việc
kq = df_model.groupby(['DevType', 'RemoteWork'])['LuongDuDoan'].mean().reset_index()
kq['LuongDuDoan'] = kq['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")

# Đổi tên cột và sắp xếp
kq = kq.rename(columns={
    'DevType': 'Ngành nghề',
    'RemoteWork': 'Hình thức làm việc',
    'LuongDuDoan': 'Thu nhập dự đoán'
})

# In kết quả
print("Dự đoán thu nhập trung bình theo ngành nghề với hình thức làm việc:")
kq


Dự đoán thu nhập trung bình theo ngành nghề với hình thức làm việc:


,Ngành nghề,Hình thức làm việc,Thu nhập dự đoán
0,Back-End Developer,Hybrid,"1,871,199,568.00 USD"
1,Back-End Developer,In-person,"1,069,478,186.00 USD"
2,Back-End Developer,Remote,"79,539,668.00 USD"
3,Embedded Developer,Hybrid,"12,278,639.00 USD"
4,Embedded Developer,In-person,"5,585,453.00 USD"
5,Embedded Developer,Remote,"29,393,415.00 USD"
6,Front-End Developer,Hybrid,"111,895,439.00 USD"
7,Front-End Developer,In-person,"1,390,777,086.00 USD"
8,Front-End Developer,Remote,"45,542,501.00 USD"
9,Full-Stack Developer,Hybrid,"88,317,155.00 USD"


In [319]:
# Chuyển đổi WorkExp sang số
def kinh_nghiem(giatri):
    if giatri == 'More than 50 years':
        return 51
    elif giatri == 'Less than 1 year':
        return 0.5
    try:
        return float(giatri)
    except:
        return None

# Lấy dữ liệu cần thiết
df_model = df[['DevType', 'Country', 'EdLevel', 'WorkExp', 'CompTotal', 'RemoteWork']].copy()

# Chuyển WorkExp sang số và loại giá trị thiếu
df_model['WorkExp'] = df_model['WorkExp'].apply(kinh_nghiem)
df_model = df_model.dropna(subset=['WorkExp', 'CompTotal'])

# Giữ lại mức lương hợp lý
df_model = df_model[(df_model['CompTotal'] > 1000) & (df_model['CompTotal'] < 50000000000000000000000)]

# Lọc quốc gia có >= 30 người
country_counts = df_model['Country'].value_counts()
quoc_gia_du = country_counts[country_counts >= 30].index
df_model = df_model[df_model['Country'].isin(quoc_gia_du)]

# Tách X và y
X = df_model.drop(columns='CompTotal')
y = df_model['CompTotal']

# Tiền xử lý
cot_so = ['WorkExp']
cot_phan_loai = ['DevType', 'Country', 'EdLevel', 'RemoteWork']

xu_ly = ColumnTransformer([
    ('so', SimpleImputer(strategy='mean'), cot_so),
    ('phan_loai', Pipeline([
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('encode', OneHotEncoder(handle_unknown='ignore'))
    ]), cot_phan_loai)
])

# Pipeline mô hình
pipeline = Pipeline([
    ('xu_ly', xu_ly),
    ('model', LinearRegression())
])

# Train model với random_state 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Dự đoán lương
df_model['LuongDuDoan'] = pipeline.predict(X)

# Lọc dự đoán bất thường
df_model = df_model[(df_model['LuongDuDoan'] > 1000) & (df_model['LuongDuDoan'] < 5000000000000000000000)]

# Tổng hợp kết quả theo quốc gia
kq_quoc_gia = df_model.groupby('Country')['LuongDuDoan'].mean().reset_index()
kq_quoc_gia['LuongDuDoan'] = kq_quoc_gia['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")
kq_quoc_gia = kq_quoc_gia.rename(columns={'LuongDuDoan': 'Thu nhập dự đoán'})

print("Dự đoán thu nhập trung bình theo quốc gia:")
kq_quoc_gia

Dự đoán thu nhập trung bình theo quốc gia:


,Country,Thu nhập dự đoán
0,Argentina,"15,136,418.00 USD"
1,Australia,"18,453,014.00 USD"
2,Austria,"17,282,240.00 USD"
3,Bangladesh,"20,118,144.00 USD"
4,Belgium,"15,771,994.00 USD"
5,Brazil,"14,481,311.00 USD"
6,Bulgaria,"18,420,291.00 USD"
7,Canada,"15,589,104.00 USD"
8,Colombia,"87,540,385.00 USD"
9,Croatia,"19,543,265.00 USD"


In [320]:
# Tổng hợp kết quả theo số năm kinh nghiệm
kq_kinh_nghiem = df_model.groupby('WorkExp')['LuongDuDoan'].mean().reset_index()
kq_kinh_nghiem['LuongDuDoan'] = kq_kinh_nghiem['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")
kq_kinh_nghiem = kq_kinh_nghiem.rename(columns={'WorkExp': 'Kinh nghiệm (năm)', 'LuongDuDoan': 'Thu nhập dự đoán'})

print("Dự đoán thu nhập trung bình theo số năm kinh nghiệm:")
kq_kinh_nghiem


Dự đoán thu nhập trung bình theo số năm kinh nghiệm:


,Kinh nghiệm (năm),Thu nhập dự đoán
0,0.0,"15,618,993.00 USD"
1,1.0,"98,685,251.00 USD"
2,2.0,"131,918,226.00 USD"
3,3.0,"78,566,462.00 USD"
4,4.0,"129,960,713.00 USD"
5,5.0,"101,367,262.00 USD"
6,6.0,"116,746,140.00 USD"
7,7.0,"167,061,920.00 USD"
8,8.0,"62,394,802.00 USD"
9,9.0,"42,906,674.00 USD"


In [321]:
# Tổng hợp kết quả theo học vấn
kq_hoc_van = df_model.groupby('EdLevel')['LuongDuDoan'].mean().reset_index()
kq_hoc_van['LuongDuDoan'] = kq_hoc_van['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")
kq_hoc_van = kq_hoc_van.rename(columns={'EdLevel': 'Trình độ học vấn', 'LuongDuDoan': 'Thu nhập dự đoán'})

print("Dự đoán thu nhập trung bình theo trình độ học vấn:")
kq_hoc_van


Dự đoán thu nhập trung bình theo trình độ học vấn:


,Trình độ học vấn,Thu nhập dự đoán
0,Associate,"126,453,231.00 USD"
1,Bachelor,"73,180,226.00 USD"
2,HighSchool,"82,123,909.00 USD"
3,Master,"70,835,260.00 USD"
4,No Degree,"46,575,505.00 USD"
5,Other,"16,298,592.00 USD"
6,Professional,"53,971,639.00 USD"


In [322]:
# Tạo lại df_model
df_model = df[['DevType', 'Country', 'EdLevel', 'WorkExp', 'CompTotal', 'RemoteWork', 'OrgSize']].copy()

# Chuyển WorkExp sang số
def kinh_nghiem(giatri):
    if giatri == 'More than 50 years':
        return 51
    elif giatri == 'Less than 1 year':
        return 0.5
    try:
        return float(giatri)
    except:
        return None

df_model['WorkExp'] = df_model['WorkExp'].apply(kinh_nghiem)
df_model = df_model.dropna(subset=['WorkExp', 'CompTotal'])
df_model = df_model[(df_model['CompTotal'] < 5000000000000000000) & (df_model['CompTotal'] > 1000)]

# Chỉ giữ quốc gia có đủ mẫu
quoc_gia_du = df_model['Country'].value_counts()[lambda x: x >= 30].index
df_model = df_model[df_model['Country'].isin(quoc_gia_du)]

# Dự đoán lại lương
X = df_model.drop(columns='CompTotal')
y = df_model['CompTotal']
df_model['LuongDuDoan'] = pipeline.predict(X)

# Tổng hợp theo OrgSize
kq_quy_mo = df_model.groupby('OrgSize')['LuongDuDoan'].mean().reset_index()
kq_quy_mo['LuongDuDoan'] = kq_quy_mo['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")
kq_quy_mo = kq_quy_mo.rename(columns={
    'OrgSize': 'Quy mô công ty',
    'LuongDuDoan': 'Thu nhập dự đoán'
})

print("Dự đoán thu nhập trung bình theo quy mô công ty:")
kq_quy_mo


Dự đoán thu nhập trung bình theo quy mô công ty:


,Quy mô công ty,Thu nhập dự đoán
0,Enterprise,"4,935,922.00 USD"
1,Large,"7,787,838.00 USD"
2,Medium,"25,023,642.00 USD"
3,Other,"70,991,833.00 USD"
4,Small,"37,025,425.00 USD"
